### Context:

* 20:29 Pod activated
* 20:29 basal 1.25 U/hr
* 20:57 temp basal 2.43 U/hr (95% more) - 1 hour duration
* 21.57 basal 1.25 U/hr


In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import dateutil

def parse(line):
    elems = line.split(' ')
    
    obj = {
        'timestamp': dateutil.parser.parse(elems[0])
    }
    for elem in elems[1:]: 
        (key,v) = elem.split(':')
        obj[key] = v
        
    return obj

def extract_byte(x, index):
    def byte_mapper(x):
        if not isinstance(x, basestring):
            return None
        if len(x) > 0:
            return ord(x.decode('hex')[index])
        else:
            return None
    return x.map(byte_mapper)

lines = open('packets.txt', "rt").readlines()
column_order = ['timestamp', 'ID1', 'PTYPE', 'SEQ', 'ID2', 'CON', 'B9', 'BLEN', 'MTYPE', 'BODY', 'CRC']
packets = pd.DataFrame(map(lambda x: parse(x), lines), columns=column_order).set_index('timestamp')
packets.head()

,ID1,PTYPE,SEQ,ID2,CON,B9,BLEN,MTYPE,BODY,CRC
timestamp,,,,,,,,,,
2016-06-17 20:23:30.966899,1f014829,POD,25,1f014829,NaN,28,10,1d18,02a82800002b7bff8134,45\n
2016-06-17 20:24:44.634776,1f014829,POD,28,1f014829,NaN,30,10,1d18,02a82800002b7fff0360,ea\n
2016-06-17 20:25:53.328967,1f014829,POD,31,1f014829,NaN,38,10,1d18,02a8a800002b87ff037c,f2\n
2016-06-17 20:48:44.551830,1f014829,POD,02,1f014829,NaN,00,10,1d18,02ad2800002bdfff009c,df\n
2016-06-17 20:48:47.828112,1f014829,POD,05,1f014829,NaN,08,10,1d18,02ad2800002bdfff83c5,cf\n


In [2]:
# Drop packets that have incorrect body length; likely corrupt packets
packets['blen_actual'] = packets['BODY'].map(lambda x: len(x) / 2, 'ignore').fillna(0).astype(np.int)
packets['BLEN'] = packets['BLEN'].map(lambda x: int(x), 'ignore').fillna(0)
ok_big_packets = (packets['blen_actual'] == 23) & (packets['BLEN'] > 23)
no_body_packets = (packets['BLEN'] == 0)
ok_small_packets = (packets['blen_actual'] == packets['BLEN']) & (packets['blen_actual'] > 0)
packets = packets.loc[ok_big_packets | no_body_packets | ok_small_packets]
packets = packets.drop('blen_actual', axis=1)

# Drop sequential duplicates
packets = packets[packets["SEQ"].shift(-1) != packets["SEQ"]]


### Look for body crc
(body crc is different from the 'CRC' column (packet crc) in the dataframe)

We're focused on the last two bytes of the body, as they seem to change like a crc.

* byte -2: bit 7 and bits 1:0 toggle, but 6:2 are always 0
* byte -1: two quiet bits: 6 and 0

In [3]:
# Let's compare packets of the same length, first
packets['BLEN'].value_counts()

0.0      326
10.0     291
3.0      259
31.0       5
24.0       3
12.0       3
205.0      2
21.0       2
6.0        2
126.0      1
50.0       1
32.0       1
29.0       1
23.0       1
Name: BLEN, dtype: int64

### xor each packet against all others, to identify bits that change (bit_deltas)

In [105]:
list(range(1,10))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [118]:
import itertools

# Compare packets
body_packets = packets.loc[packets['BLEN'] == 10]

# Build raw packet data of everything after sequence, as sequence doesn't appear to
# affect the crc
blen_hex = body_packets["BLEN"].map(lambda x: chr(int(x)).encode('hex'))
raw_hex_data = body_packets["ID2"] + body_packets["B9"] + blen_hex + body_packets["MTYPE"] + body_packets["BODY"]

print raw_hex_data.head()
raw_data = raw_hex_data.map(lambda x: x.decode('hex'))

def xor_strings(s1,s2):
    xored_bytes = [ord(a) ^ ord(b) for a,b in zip(s1,s2)]
    return [s1.encode('hex'),s2.encode('hex')] + list(np.unpackbits(np.array(xored_bytes, dtype=np.uint8)))

combinations = map(lambda x: xor_strings(x[0], x[1]), itertools.combinations(list(raw_data), 2))
bit_deltas = pd.DataFrame(combinations)

#
crc_split_point = 128
bit_cols = list(range(0,len(combinations[0])-2))
body_columns = map(lambda x: ('body', x), bit_cols[:crc_split_point])
crc_columns = map(lambda x: ('crc', x), bit_cols[crc_split_point:])
                
bit_deltas.columns = pd.MultiIndex.from_tuples([('data', 'p1'), ('data', 'p2')] + body_columns + crc_columns)
bit_deltas['num_body_deltas'] = split_bit_deltas['body'].sum(axis=1)
bit_deltas['num_crc_deltas'] = split_bit_deltas['crc'].sum(axis=1)

bit_deltas.head()

timestamp
2016-06-17 20:23:30.966899    1f014829280a1d1802a82800002b7bff8134
2016-06-17 20:24:44.634776    1f014829300a1d1802a82800002b7fff0360
2016-06-17 20:25:53.328967    1f014829380a1d1802a8a800002b87ff037c
2016-06-17 20:48:44.551830    1f014829000a1d1802ad2800002bdfff009c
2016-06-17 20:48:47.828112    1f014829080a1d1802ad2800002bdfff83c5
dtype: object


data                                        \
                                     p1                                    p2   
0  1f014829280a1d1802a82800002b7bff8134  1f014829300a1d1802a82800002b7fff0360   
1  1f014829280a1d1802a82800002b7bff8134  1f014829380a1d1802a8a800002b87ff037c   
2  1f014829280a1d1802a82800002b7bff8134  1f014829000a1d1802ad2800002bdfff009c   
3  1f014829280a1d1802a82800002b7bff8134  1f014829080a1d1802ad2800002bdfff83c5   
4  1f014829280a1d1802a82800002b7bff8134  1f014829100a1d1802ad2800002bdfff014b   

  body                                                                       \
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23   
0    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
3    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
4    0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

                                                                              \
  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48   
0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0   
2  0  0  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0   
3  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
4  0  0  0  0  0  0  0  0  0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0   

                                                                              \
  49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73   
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

                                                                              \
  74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98   
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
2  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
3  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
4  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   

                                                                              \
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117   
0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   
1  0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1   1   1   1   
2  0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   1   0   0   1   
3  0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   1   0   0   1   
4  0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   1   0   0   1   

                                          crc                                  \
  118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136   
0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   1   0   0   
1   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   1   0   0   
2   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   1   1   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   1   
4   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   

                              num_body_deltas num_crc_deltas  
  137 138 139 140 141 142 143         

In [120]:

# This selects the number of 'collisions', or different frames with same crc.
# It could be used to identify bits not part of crc
#split_bit_deltas[(split_bit_deltas["num_body_deltas"] != 0) & (split_bit_deltas["num_crc_deltas"] == 0)]

# Look at low change rows
pd.set_option('display.max_colwidth',1000)
pd.options.display.max_columns = 200
ones = bit_deltas[(bit_deltas["num_body_deltas"] == 1)]
ones.loc[ones[('body',34)] == 1]

data  \
                                         p1   
2295   1f0148292c0a1d1802af2800002c07ff026c   
2577   1f014829340a1d1802af2800002c07ff80e2   
2858   1f0148293c0a1d1802af2800002c07ff03bb   
3138   1f014829040a1d1802af2800002c07ff03e9   
28170  1f02d5af180a1d1801d9180000102fff020a   
28337  1f02d5af200a1d1801d9180000102fff0258   
29157  1f02d5af140a1d1801d9180000102fff014b   
29318  1f02d5af1c0a1d1801d9180000102fff8212   
29478  1f02d5af240a1d1801d9180000102fff8240   
30263  1f02d5af0c0a1d1801d91800001033ff82ff   
30271  1f02d5af0c0a1d1801d91800001033ff82ff   
30279  1f02d5af0c0a1d1801d91800001033ff82ff   
30287  1f02d5af0c0a1d1801d91800001033ff82ff   
30295  1f02d5af0c0a1d1801d91800001033ff82ff   
30417  1f02d5af140a1d1801d91800001033ff0071   
30425  1f02d5af140a1d1801d91800001033ff0071   
30433  1f02d5af140a1d1801d91800001033ff0071   
30441  1f02d5af140a1d1801d91800001033ff0071   
30449  1f02d5af140a1d1801d91800001033ff0071   
30570  1f02d5af1c0a1d1801d91800001033ff8328   
30578  1f02d5af1c0a1d1801d91800001033ff8328   
30586  1f02d5af1c0a1d1801d91800001033ff8328   
30594  1f02d5af1c0a1d1801d91800001033ff8328   
30602  1f02d5af1c0a1d1801d91800001033ff8328   
30722  1f02d5af240a1d1801d91800001033ff837a   
30730  1f02d5af240a1d1801d91800001033ff837a   
30738  1f02d5af240a1d1801d91800001033ff837a   
30746  1f02d5af240a1d1801d91800001033ff837a   
30754  1f02d5af240a1d1801d91800001033ff837a   
30873  1f02d5af2c0a1d1801d91800001033ff0023   
...                                     ...   
41967  1f014829340a1d190329380000360fdd8151   
41975  1f014829340a1d190329380000360fdd8151   
41983  1f014829340a1d190329380000360fdd8151   
41988  1f0148293c0a1d190329380000360fdd0208   
41996  1f0148293c0a1d190329380000360fdd0208   
42004  1f0148293c0a1d190329380000360fdd0208   
42008  1f014829040a1d190329380000360fdd025a   
42016  1f014829040a1d190329380000360fdd025a   
42027  1f0148290c0a1d190329380000360fdd8103   
42035  1f0148290c0a1d190329380000360fdd8103   
42045  1f014829140a1d190329380000360fdd038d   
42053  1f014829140a1d190329380000360fdd038d   
42062  1f0148291c0a1d190329380000360fdd80d4   
42070  1f0148291c0a1d190329380000360fdd80d4   
42078  1f014829240a1d190329380000360fdd8086   
42086  1f014829240a1d190329380000360fdd8086   
42093  1f0148292c0a1d190329380000360fdd03df   
42101  1f0148292c0a1d190329380000360fdd03df   
42107  1f014829340a1d190329380000360fdd8151   
42115  1f014829340a1d190329380000360fdd8151   
42120  1f0148293c0a1d190329380000360fdd0208   
42128  1f0148293c0a1d190329380000360fdd0208   
42132  1f014829040a1d190329380000360fdd025a   
42143  1f0148290c0a1d190329380000360fdd8103   
42153  1f014829140a1d190329380000360fdd038d   
42162  1f0148291c0a1d190329380000360fdd80d4   
42170  1f014829240a1d190329380000360fdd8086   
42177  1f0148292c0a1d190329380000360fdd03df   
42183  1f014829340a1d190329380000360fdd8151   
42188  1f0148293c0a1d190329380000360fdd0208   

                                            body                             \
                                         p2    0  1  2  3  4  5  6  7  8  9   
2295   1f0148290c0a1d1802af2800002c07ff80b0    0  0  0  0  0  0  0  0  0  0   
2577   1f014829140a1d1802af2800002c07ff023e    0  0  0  0  0  0  0  0  0  0   
2858   1f0148291c0a1d1802af2800002c07ff8167    0  0  0  0  0  0  0  0  0  0   
3138   1f014829240a1d1802af2800002c07ff8135    0  0  0  0  0  0  0  0  0  0   
28170  1f02d5af380a1d1801d9180000102fff80d6    0  0  0  0  0  0  0  0  0  0   
28337  1f02d5af000a1d1801d9180000102fff8084    0  0  0  0  0  0  0  0  0  0   
29157  1f02d5af340a1d1801d9180000102fff8397    0  0  0  0  0  0  0  0  0  0   
29318  1f02d5af3c0a1d1801d9180000102fff00ce    0  0  0  0  0  0  0  0  0  0   
29478  1f02d5af040a1d1801d9180000102fff009c    0  0  0  0  0  0  0  0  0  0   
30263  1f02d5af2c0a1d1801d91800001033ff0023    0  0  0  0  0  0  0  0  0  0   
30271  1f02d5af2c0a1d1801d91800001033ff0023    0  0  0  0  0  0  0  0  0  0   
30279  1f02d5af2c0a1d1801d91800001033ff0023    0  

In [50]:
# Split bit_deltas in two; crc bits and data bits

body_bds = bit_deltas[bit_deltas.columns[:128]].copy()
crc_bds = bit_deltas[bit_deltas.columns[128:]].copy()
body_bds["sum"] = body_bds.sum(axis=1)
b

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,sum
22692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39284,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12063,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12062,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13754,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33817,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
